## Association Rules - Old vs. New Algorithm

### Dataset: eCommerce_purchase_history_from_cosmetics_store

#### 1 Investigating Dataset

#### 1.1 Description
Kaggle Dataset: https://www.kaggle.com/datasets/mkechinov/ecommerce-events-history-in-cosmetics-shop <br>
The Dataset is Open-Source <br>
This file contains behavior data for 5 months (Oct 2019 – Feb 2020) from a medium cosmetics online store.<br>

In [1]:
import os
import pandas as pd
import glob

In [2]:
import glob
import os

# setting the path for joining multiple files
files = os.path.join(os.getcwd(), "datasets/eCommerce_purchase_history_from_cosmetics_store/*.csv")

# list of merged files returned
files = glob.glob(files)

# joining files with concat and read_csv
data_raw = pd.concat(map(pd.read_csv, files), ignore_index=True).iloc[:,1:].rename(columns={"user_session":"order_id"})

data_raw


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,order_id
0,2019-12-01 00:17:50 UTC,purchase,4554,1487580010796548183,NaN,runail,0.38,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9
1,2019-12-01 00:17:50 UTC,purchase,4600,1487580010796548183,NaN,runail,0.38,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9
2,2019-12-01 00:17:50 UTC,purchase,4607,1487580010796548183,NaN,runail,0.38,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9
3,2019-12-01 00:17:50 UTC,purchase,4640,1487580011157258342,NaN,runail,0.37,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9
4,2019-12-01 00:17:50 UTC,purchase,4689,1487580010796548183,NaN,runail,0.38,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9
...,...,...,...,...,...,...,...,...,...
1287002,2020-01-31 23:58:26 UTC,purchase,5730206,1487580005092295511,NaN,NaN,10.32,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e
1287003,2020-01-31 23:58:26 UTC,purchase,5861342,1487580010645553231,NaN,NaN,1.90,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e
1287004,2020-01-31 23:58:26 UTC,purchase,5867185,1487580007910867929,NaN,staleks,11.43,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e
1287005,2020-01-31 23:58:26 UTC,purchase,5877766,1605161575889502297,NaN,NaN,4.76,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e


In [3]:
# Quantity is always 1
grouped_mutliple_items = data_raw.groupby(['order_id', 'product_id']).agg({'user_id': ['count']})

#grouped_mutliple_items[grouped_mutliple_items["quantity"]>1]

grouped_mutliple_items.columns = ["user_id"]
grouped_mutliple_items.rename(columns={"user_id":"quantity"},inplace=True)
grouped_mutliple_items = grouped_mutliple_items.reset_index()
grouped_mutliple_items = grouped_mutliple_items[grouped_mutliple_items["quantity"]>1].sort_values("quantity",ascending=False)

In [4]:
grouped_mutliple_items

,order_id,product_id,quantity
368420,49bc852c-cbc7-4d84-9a2c-8144205b571a,5751422,10
368421,49bc852c-cbc7-4d84-9a2c-8144205b571a,5809910,10
1252940,fb5a2f8f-837c-4e50-9b3d-bdcbdc75315e,5778113,9
368422,49bc852c-cbc7-4d84-9a2c-8144205b571a,5809912,9
368414,49bc852c-cbc7-4d84-9a2c-8144205b571a,5304,9
...,...,...,...
435978,570ac7f0-7bc2-7587-5c70-388c76d73451,5809303,2
435979,570ac7f0-7bc2-7587-5c70-388c76d73451,5841799,2
435980,570ac7f0-7bc2-7587-5c70-388c76d73451,5856952,2
435981,570ac7f0-7bc2-7587-5c70-388c76d73451,5879128,2


In [5]:
# Mutiple items per transaction

import seaborn as sns

mutliple_items_per_transaction = grouped_mutliple_items["quantity"].value_counts()
mutliple_items_per_transaction.rename_axis("frequency")

# lenth_of_transactions

total = mutliple_items_per_transaction.values.sum()

import plotly.express as px
fig = px.bar(mutliple_items_per_transaction, x=mutliple_items_per_transaction.index, y=mutliple_items_per_transaction.values/total,width=400, height=400)
fig.update_traces(width=0.5)
fig.update_layout(
    xaxis_title="Multiple items per transaction",
    yaxis_tickformat ='0%',
    xaxis_tickformat = ',d',
    yaxis_title="Distribution",
    legend_title="Multiple items per transaction distribution",)

fig.show()


In [6]:
# group by Transaction

import math
import datetime

data_raw["date"] = pd.to_datetime(data_raw['event_time']).dt.date
max_date = data_raw["date"].max()
data_raw["margin"] = 0.1
data_raw["profit"] = data_raw["price"] * data_raw["margin"]
max_profit = data_raw["profit"].max()

data = data_raw.groupby("order_id",dropna=True)["product_id","date","price","margin"].agg(lambda x: list(x))

data

C:\Users\david\AppData\Local\Temp/ipykernel_1512/4105145759.py:12: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,product_id,date,price,margin
order_id,,,,
000013d6-68a4-40cf-9452-6577dbfab515,"[5889104, 5878254, 5875502, 5859210, 5739125, ...","[2019-10-23, 2019-10-23, 2019-10-23, 2019-10-2...","[4.76, 5.56, 1.75, 0.48, 4.4, 5.22, 0.48, 5.08...","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]"
00008836-6d96-4580-8527-173d4e1b2889,"[5730402, 5853682, 5839703, 5839663, 5839660, ...","[2019-10-02, 2019-10-02, 2019-10-02, 2019-10-0...","[7.86, 3.63, 1.73, 2.37, 2.37, 1.73, 1.73, 1.4...","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ..."
0000d04d-3516-42ea-bdb9-584e92ce619e,"[5809100, 5809101, 5854897, 5888573]","[2020-02-15, 2020-02-15, 2020-02-15, 2020-02-15]","[1.59, 1.59, 0.32, 3.33]","[0.1, 0.1, 0.1, 0.1]"
0001981a-ee7c-4542-bf81-2cea59d963ab,"[5839084, 5788435, 5743913, 5743912, 5677989, ...","[2019-10-05, 2019-10-05, 2019-10-05, 2019-10-0...","[4.86, 0.79, 3.97, 3.97, 8.73, 1.43]","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]"
000277a9-30da-4bb2-b5ad-96ef24483af8,"[5815567, 5815568, 5815569, 5815570, 5815571]","[2019-12-14, 2019-12-14, 2019-12-14, 2019-12-1...","[0.71, 0.71, 0.71, 0.71, 0.71]","[0.1, 0.1, 0.1, 0.1, 0.1]"
...,...,...,...,...
fffee574-c72f-03cf-4d2a-5188047c4cef,"[5560968, 5857357, 5695534, 5723501, 5734268, ...","[2019-12-24, 2019-12-24, 2019-12-24, 2019-12-2...","[3.0, 2.78, 3.33, 2.62, 3.97, 6.19, 3.33, 3.0,...","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]"
ffff78e3-1e8f-4eda-bb42-95749ab4f42e,"[5686083, 5710585, 5776129, 5776130, 5776132]","[2020-02-21, 2020-02-21, 2020-02-21, 2020-02-2...","[0.79, 1.43, 2.7, 1.98, 2.7]","[0.1, 0.1, 0.1, 0.1, 0.1]"
ffff8529-58c1-4ec8-933e-77f535718061,"[5875389, 5859403, 5839651, 5752164, 5727536, ...","[2019-10-13, 2019-10-13, 2019-10-13, 2019-10-1...","[2.05, 2.37, 2.37, 3.97, 2.7, 1.73]","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]"


In [7]:
# Length of transactions

import seaborn as sns

length_of_transactions = data["product_id"].apply(len).value_counts()
length_of_transactions = length_of_transactions.sort_index(ascending=True).head(50) # For graph only shop top 50 
length_of_transactions.rename_axis("frequency")


total = length_of_transactions.sum()

import plotly.express as px
fig = px.bar(length_of_transactions, x=length_of_transactions.index, y=length_of_transactions.values/total)
fig.update_traces(width=0.5)
fig.update_layout(
    xaxis_title="Number of items per transaction",
    yaxis_tickformat =',.0%',
    yaxis_title="Distribution (%)",
    legend_title="Transaction Length Dsitribution",)

fig.show()

In [8]:
unique_items = data_raw[["product_id","category_id","category_code","brand","price"]].drop_duplicates(subset = "product_id").astype("str")


unique_items

,product_id,category_id,category_code,brand,price
0,4554,1487580010796548183,nan,runail,0.38
1,4600,1487580010796548183,nan,runail,0.38
2,4607,1487580010796548183,nan,runail,0.38
3,4640,1487580011157258342,nan,runail,0.37
4,4689,1487580010796548183,nan,runail,0.38
...,...,...,...,...,...
1285867,5913479,1542195323827388674,nan,dermacol,13.97
1285868,5915330,1542195323827388674,nan,dermacol,19.05
1286757,5923045,1487580007675986893,nan,nan,1.43
1286948,5889114,2134354342373753638,nan,nan,3.49


In [9]:
unique_items["price"].astype("float").median()
unique_items["price"].astype("float").mean()

7.056675331682017

In [10]:
data_raw["date"] = pd.to_datetime(data_raw['event_time']).dt.date
max_date = data_raw["date"].max()
data_raw["margin"] = 0.1
data_raw["profit"] = data_raw["price"] * data_raw["margin"]
max_profit = data_raw["profit"].max()

data = data_raw.groupby("order_id",dropna=True)["product_id","date","price","margin"].agg(lambda x: list(x))

data

C:\Users\david\AppData\Local\Temp/ipykernel_1512/16707732.py:7: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,product_id,date,price,margin
order_id,,,,
000013d6-68a4-40cf-9452-6577dbfab515,"[5889104, 5878254, 5875502, 5859210, 5739125, ...","[2019-10-23, 2019-10-23, 2019-10-23, 2019-10-2...","[4.76, 5.56, 1.75, 0.48, 4.4, 5.22, 0.48, 5.08...","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]"
00008836-6d96-4580-8527-173d4e1b2889,"[5730402, 5853682, 5839703, 5839663, 5839660, ...","[2019-10-02, 2019-10-02, 2019-10-02, 2019-10-0...","[7.86, 3.63, 1.73, 2.37, 2.37, 1.73, 1.73, 1.4...","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ..."
0000d04d-3516-42ea-bdb9-584e92ce619e,"[5809100, 5809101, 5854897, 5888573]","[2020-02-15, 2020-02-15, 2020-02-15, 2020-02-15]","[1.59, 1.59, 0.32, 3.33]","[0.1, 0.1, 0.1, 0.1]"
0001981a-ee7c-4542-bf81-2cea59d963ab,"[5839084, 5788435, 5743913, 5743912, 5677989, ...","[2019-10-05, 2019-10-05, 2019-10-05, 2019-10-0...","[4.86, 0.79, 3.97, 3.97, 8.73, 1.43]","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]"
000277a9-30da-4bb2-b5ad-96ef24483af8,"[5815567, 5815568, 5815569, 5815570, 5815571]","[2019-12-14, 2019-12-14, 2019-12-14, 2019-12-1...","[0.71, 0.71, 0.71, 0.71, 0.71]","[0.1, 0.1, 0.1, 0.1, 0.1]"
...,...,...,...,...
fffee574-c72f-03cf-4d2a-5188047c4cef,"[5560968, 5857357, 5695534, 5723501, 5734268, ...","[2019-12-24, 2019-12-24, 2019-12-24, 2019-12-2...","[3.0, 2.78, 3.33, 2.62, 3.97, 6.19, 3.33, 3.0,...","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]"
ffff78e3-1e8f-4eda-bb42-95749ab4f42e,"[5686083, 5710585, 5776129, 5776130, 5776132]","[2020-02-21, 2020-02-21, 2020-02-21, 2020-02-2...","[0.79, 1.43, 2.7, 1.98, 2.7]","[0.1, 0.1, 0.1, 0.1, 0.1]"
ffff8529-58c1-4ec8-933e-77f535718061,"[5875389, 5859403, 5839651, 5752164, 5727536, ...","[2019-10-13, 2019-10-13, 2019-10-13, 2019-10-1...","[2.05, 2.37, 2.37, 3.97, 2.7, 1.73]","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]"


#### 3.1.1 Classical Association Rules

In [11]:
import modified_fp_growth_algorithm.modified_fp_growth_latest as mod_fp_growth

rules = mod_fp_growth.fpgrowthFromDataFrame(\
    data,
    minSupRatio=0.01,
    maxSupRatio=1,
    minConf=0,
    item_col=1
    )
    
rules.to_excel("fp_groth_out.xlsx",index=False) 
rules

,antecedent,sup_antecedent,consequent,sup_consequent,antecedent&consequent,sup_ant&cons,sup_perc_ant&cons,confidence,lift,improvement
0,[],NA,[5809910],7458,[5809910],7458,0.047925,NA,NA,NA
1,[],NA,[5854897],4593,[5854897],4593,0.029515,NA,NA,NA
2,[],NA,[5700037],3623,[5700037],3623,0.023282,NA,NA,NA
3,[],NA,[5802432],3486,[5802432],3486,0.022401,NA,NA,NA
4,[],NA,[5751422],3457,[5751422],3457,0.022215,NA,NA,NA
5,[],NA,[5809912],3272,[5809912],3272,0.021026,NA,NA,NA
6,[],NA,[5815662],3206,[5815662],3206,0.020602,NA,NA,NA
7,[],NA,[5304],3102,[5304],3102,0.019934,NA,NA,NA
8,[],NA,[5751383],2912,[5751383],2912,0.018713,NA,NA,NA
9,[],NA,[5849033],2753,[5849033],2753,0.017691,NA,NA,NA


In [12]:
# A minSupRatio of 0.007 means, in case of profitability, that it represents 0.007 of total revenues rspectively profit. We made the profit fo each product equal.
# Let's oberserve what kind of items we have found.


rules_exploded = rules.explode("antecedent&consequent")[["antecedent&consequent","sup_ant&cons"]]
rules_exploded.insert(0,"rules_index",rules_exploded.index)


rules_labeled = pd.merge(rules_exploded,unique_items, how='inner', left_on = 'antecedent&consequent', right_on = 'product_id').sort_values("rules_index").reset_index(drop=True)


rules_labeled.to_excel("rules_labeled.xlsx",index=False)

rules_labeled = rules_labeled.drop('rules_index', axis=1)
rules_labeled

,antecedent&consequent,sup_ant&cons,product_id,category_id,category_code,brand,price
0,5809910,7458,5809910,1602943681873052386,nan,grattol,5.24
1,5854897,4593,5854897,1487580009445982239,nan,irisk,0.32
2,5700037,3623,5700037,1487580009286598681,nan,runail,0.4
3,5802432,3486,5802432,1487580009286598681,nan,nan,0.32
4,5751422,3457,5751422,1487580005268456287,nan,uno,10.95
5,5809912,3272,5809912,1602943681873052386,nan,grattol,5.24
6,5815662,3206,5815662,1487580006317032337,nan,nan,0.92
7,5304,3102,5304,1487580009471148064,nan,runail,0.32
8,5751383,2912,5751383,1487580005092295511,nan,uno,10.32
9,5849033,2753,5849033,1487580005092295511,nan,uno,10.32


#### 3.1.2 Classical Association Rules (Limitations on transaction length as at least 2)

In [16]:
import modified_fp_growth_algorithm.modified_fp_growth_latest as mod_fp_growth

data = data[data["product_id"].apply(len)<5]

rules = mod_fp_growth.fpgrowthFromDataFrame(\
    data,
    minSupRatio=0.001,
    maxSupRatio=1,
    minConf=0,
    item_col=1,
    minTransactionLength=2,
    maxTransactionLength=10
    )
    
rules.to_excel("fp_groth_out.xlsx",index=False) 
rules

,antecedent,sup_antecedent,consequent,sup_consequent,antecedent&consequent,sup_ant&cons,sup_perc_ant&cons,confidence,lift,improvement
0,[5809910],1642,[5809912],466,"[5809910, 5809912]",231,0.003565,0.140682,19.563868,0.133491
1,[5751422],596,[5751383],595,"[5751383, 5751422]",175,0.002700,0.293624,31.979866,0.284443
2,[5809910],1642,[5809911],270,"[5809910, 5809911]",161,0.002484,0.098051,23.533730,0.093885
3,[5814516],183,[5814517],170,"[5814516, 5814517]",148,0.002284,0.808743,308.292896,0.806120
4,[5677043],447,[5697463],196,"[5697463, 5677043]",129,0.001991,0.288591,95.417477,0.285566
5,[5823667],241,[5819894],125,"[5819894, 5823667]",113,0.001744,0.468880,243.082224,0.466951
6,[5751422],596,[5849033],465,"[5849033, 5751422]",93,0.001435,0.156040,21.746309,0.148865
7,[5809910],1642,[5816170],233,"[5816170, 5809910]",90,0.001389,0.054811,15.244572,0.051216
8,[4185],228,[3762],147,"[3762, 4185]",88,0.001358,0.385965,170.150137,0.383697
9,[5528035],493,[5528034],205,"[5528035, 5528034]",84,0.001296,0.170385,53.861733,0.167222


In [19]:
rules_exploded = rules.explode("antecedent&consequent")[["antecedent&consequent","sup_ant&cons"]]
rules_exploded.insert(0,"rules_index",rules_exploded.index)


rules_labeled = pd.merge(rules_exploded,unique_items, how='inner', left_on = 'antecedent&consequent', right_on = 'product_id').sort_values("rules_index").reset_index(drop=True)


rules_labeled.to_excel("rules_labeled.xlsx",index=False)

rules_labeled


,rules_index,antecedent&consequent,sup_ant&cons,product_id,category_id,category_code,brand,price
0,0,5809910,231,5809910,1602943681873052386,nan,grattol,5.24
1,0,5809912,231,5809912,1602943681873052386,nan,grattol,5.24
2,1,5751383,175,5751383,1487580005092295511,nan,uno,10.32
3,1,5751422,175,5751422,1487580005268456287,nan,uno,10.95
4,2,5809910,161,5809910,1602943681873052386,nan,grattol,5.24
5,2,5809911,161,5809911,1602943681873052386,nan,grattol,5.24
6,3,5814516,148,5814516,1487580013539623112,nan,nan,13.49
7,3,5814517,148,5814517,1487580013539623112,nan,nan,12.54
8,4,5697463,129,5697463,1783999067156644376,nan,estel,12.14
9,4,5677043,129,5677043,1487580008246412266,nan,estel,13.33


#### 3.2 Profit based Association Rules (No limitations on transaction length)

In [31]:
import modified_fp_growth_algorithm.modified_fp_growth_latest as mod_fp_growth

#For better trackingI take price instead of profit as profit, because tracking ius easier and at moment I anyway just multiply by 0.1 for profit
# We see how "super efficient it is, only 6.8 seconds for 13k rules inclduing writing to Excel"

data = data[data["product_id"].apply(len)<5]

rules = mod_fp_growth.fpgrowthFromDataFrame(\
    data,
    minSupRatio=0.005,
    maxSupRatio=1,
    minConf=0,
    item_col=1,
    profit_col=3,
    max_profit = max_profit,
    profit_sensitivity = lambda x : 1 * x
    )
    
rules.to_excel("fp_groth_out.xlsx",index=False) 
rules


,antecedent,sup_antecedent,consequent,sup_consequent,antecedent&consequent,sup_ant&cons,sup_perc_ant&cons,confidence,lift,improvement,profit_associated,perc_of_total_profit,profit_associated_prev,net_change,profit_last_item,loss_by_change
0,[],NA,[5560754],187,[5560754],187,0.002886,NA,NA,NA,36261.00,0.023414,0,0,36261.00,0
1,[],NA,[89343],47,[89343],47,0.000725,NA,NA,NA,14327.88,0.009252,0,0,14327.88,0
2,[],NA,[5877454],315,[5877454],315,0.004861,NA,NA,NA,13877.76,0.008961,0,0,13877.76,0
3,[],NA,[5850281],90,[5850281],90,0.001389,NA,NA,NA,12593.00,0.008132,0,0,12593.00,0
4,[],NA,[5692527],256,[5692527],256,0.003950,NA,NA,NA,11785.57,0.007610,0,0,11785.57,0
5,[],NA,[5856186],150,[5856186],150,0.002315,NA,NA,NA,11239.06,0.007257,0,0,11239.06,0
6,[],NA,[5846437],248,[5846437],248,0.003827,NA,NA,NA,11234.78,0.007255,0,0,11234.78,0
7,[],NA,[5560756],50,[5560756],50,0.000772,NA,NA,NA,10573.74,0.006828,0,0,10573.74,0
8,[],NA,[5855507],125,[5855507],125,0.001929,NA,NA,NA,9980.46,0.006445,0,0,9980.46,0
9,[],NA,[5804820],138,[5804820],138,0.002129,NA,NA,NA,8757.90,0.005655,0,0,8757.90,0


In [22]:
rules_exploded = rules.explode("antecedent&consequent")[["antecedent&consequent","sup_ant&cons"]]
rules_exploded.insert(0,"rules_index",rules_exploded.index)


rules_labeled = pd.merge(rules_exploded,unique_items, how='inner', left_on = 'antecedent&consequent', right_on = 'product_id').sort_values("rules_index").reset_index(drop=True)


rules_labeled.to_excel("rules_labeled.xlsx",index=False)

rules_labeled = rules_labeled.drop('rules_index', axis=1)
rules_labeled


,antecedent&consequent,sup_ant&cons,product_id,category_id,category_code,brand,price
0,5560754,187,5560754,1487580006300255120,nan,strong,194.44
1,89343,47,89343,2193074740619379535,furniture.living_room.cabinet,nan,299.81
2,5877454,315,5877454,1487580006300255120,nan,jessnail,44.29
3,5850281,90,5850281,1487580006300255120,nan,marathon,137.78
4,5692527,256,5692527,1487580014009385185,nan,nan,46.02
5,5856186,150,5856186,1487580006300255120,nan,runail,75.4
6,5846437,248,5846437,1487580013950664926,nan,browxenna,45.24
7,5560756,50,5560756,1487580006300255120,nan,strong,207.94
8,5855507,125,5855507,1487580006350586771,appliances.environment.vacuum,max,79.21
9,5804820,138,5804820,1487580013950664926,nan,nan,63.33


#### 3.2 Profit based Association Rules (Limitations on transaction length as at least 2)

In [32]:
import modified_fp_growth_algorithm.modified_fp_growth_latest as mod_fp_growth

#For better trackingI take price instead of profit as profit, because tracking ius easier and at moment I anyway just multiply by 0.1 for profit
# We see how "super efficient it is, only 6.8 seconds for 13k rules inclduing writing to Excel"

data = data[data["product_id"].apply(len)<5]

rules = mod_fp_growth.fpgrowthFromDataFrame(\
    data,
    minSupRatio=0.0007,
    maxSupRatio=1,
    minTransactionLength=2,
    maxTransactionLength=10,
    minConf=0,
    item_col=1,
    profit_col=3,
    max_profit = max_profit,
    profit_sensitivity = lambda x : 1 * x
    )
    
rules.to_excel("fp_groth_out.xlsx",index=False) 
rules

,antecedent,sup_antecedent,consequent,sup_consequent,antecedent&consequent,sup_ant&cons,sup_perc_ant&cons,confidence,lift,improvement,profit_associated,perc_of_total_profit,profit_associated_prev,net_change,profit_last_item,loss_by_change
0,[5814516],183,[5814517],170,"[5814516, 5814517]",148,0.002284,0.808743,308.292896,0.806120,3833.911456,0.002476,2457.87,1376.041456,1846.125882,-470.084426
1,[5751422],596,[5751383],595,"[5751383, 5751422]",175,0.002700,0.293624,31.979866,0.284443,3720.919621,0.002403,6552.00,-2831.080379,1797.094118,-4628.174497
2,[4185],228,[3762],147,"[3762, 4185]",88,0.001358,0.385965,170.150137,0.383697,3394.591572,0.002192,4402.79,-1008.198428,1695.269116,-2703.467544
3,[5677043],447,[5697463],196,"[5697463, 5677043]",129,0.001991,0.288591,95.417477,0.285566,3277.924543,0.002117,5928.48,-2650.555457,1567.020918,-4217.576376
4,[5809910],1642,[5809912],466,"[5809910, 5809912]",231,0.003565,0.140682,19.563868,0.133491,2404.782980,0.001553,8558.51,-6153.727020,1200.753863,-7354.480883
5,[5846437],248,[5846436],39,"[5846436, 5846437]",22,0.000339,0.088710,147.403639,0.088108,1985.764992,0.001282,11234.78,-9249.015008,989.131282,-10238.146290
6,[5751422],596,[5849033],465,"[5849033, 5751422]",93,0.001435,0.156040,21.746309,0.148865,1973.919839,0.001275,6552.00,-4578.080161,951.544000,-5529.624161
7,[5809910],1642,[5809911],270,"[5809910, 5809911]",161,0.002484,0.098051,23.533730,0.093885,1678.601957,0.001084,8558.51,-6879.908043,839.430148,-7719.338191
8,[5528035],493,[5528034],205,"[5528035, 5528034]",84,0.001296,0.170385,53.861733,0.167222,1589.509588,0.001026,4648.63,-3059.120412,797.450927,-3856.571339
9,[5823970],125,[5820270],114,"[5823970, 5820270]",26,0.000401,0.208000,118.238877,0.206241,1391.193331,0.000898,3348.89,-1957.696669,694.624211,-2652.320880


In [33]:
# A minSupRatio of 0.007 means, in case of profitability, that it represents 0.007 of total revenues rspectively profit. We made the profit fo each product equal.
# Let's oberserve what kind of items we have found.


rules_exploded = rules.explode("antecedent&consequent")[["antecedent&consequent","sup_ant&cons","profit_associated","perc_of_total_profit"]]
rules_exploded.insert(0,"rules_index",rules_exploded.index)

rules_labeled =pd.merge(rules_exploded,unique_items, how="inner", left_on="antecedent&consequent",right_on="product_id").sort_values("rules_index").reset_index(drop=True)

rules_labeled.to_excel("rules_labeled.xlsx",index=False)

rules_labeled["profit_associated"] = rules_labeled["profit_associated"].astype(int)

rules_labeled.drop("category_code",axis=1)

rules_labeled.head(16)

,rules_index,antecedent&consequent,sup_ant&cons,profit_associated,perc_of_total_profit,product_id,category_id,category_code,brand,price
0,0,5814516,148,3833,0.002476,5814516,1487580013539623112,nan,nan,13.49
1,0,5814517,148,3833,0.002476,5814517,1487580013539623112,nan,nan,12.54
2,1,5751383,175,3720,0.002403,5751383,1487580005092295511,nan,uno,10.32
3,1,5751422,175,3720,0.002403,5751422,1487580005268456287,nan,uno,10.95
4,2,3762,88,3394,0.002192,3762,1487580005411062629,nan,cnd,19.37
5,2,4185,88,3394,0.002192,4185,1487580005411062629,nan,cnd,19.37
6,3,5697463,129,3277,0.002117,5697463,1783999067156644376,nan,estel,12.14
7,3,5677043,129,3277,0.002117,5677043,1487580008246412266,nan,estel,13.33
8,4,5809910,231,2404,0.001553,5809910,1602943681873052386,nan,grattol,5.24
9,4,5809912,231,2404,0.001553,5809912,1602943681873052386,nan,grattol,5.24


#### 3.3 Sensitivity Analysis

In [ ]:
import math
import datetime

rules = mod_fp_growth.fpgrowthFromDataFrame(\
    df_data_simple_withprofit,
    minSupRatio=0.03,
    maxSupRatio=1,
    minConf=0,
    item_col=1,
    profit_col=2,
    max_profit = 100,
    profit_sensitivity = lambda x : 1 * x
    ) #Only Date

rules


NameError: name 'df_data_simple_withprofit' is not defined